Codigo para transformar el corpus de entrada anotado en formato BRAT Standoff en formato IOB.


Para ello usamos el tokenizador de Spacy. Hay dos versiones posibles en español:


1.   !python -m spacy download es_core_news_md
2.   !python -m spacy download es_core_news_sm


Y por cada versión sm, md o lg, en función del tamaño del modelo. Esto se puede ver [en el siguiente enlace](https://stackoverflow.com/questions/50487495/what-is-difference-between-en-core-web-sm-en-core-web-mdand-en-core-web-lg-mod)

Establecemos conexión con Google Drive

In [ ]:
from google.colab import drive
#Enlazamos nuestro notebook en Colab con nuestro almacenamiento en Google Drive 
drive.mount('/content/drive')

Mounted at /content/drive


Instalamos spaCy

In [ ]:
# install the requirements
!pip install spacy

# Instalamos los modelos
#!python -m spacy download en_core_web_md
#!python -m spacy download en_core_news_md
#!python -m spacy download es_core_web_md
!python -m spacy download es_core_news_sm # El que recomienda spacy por defecto
#!python -m spacy download es_core_news_md # El primero usado por mi
#!python -m spacy download es_core_news_lg #


# Concatenar ann data
# No nos sirve, finalmente no hacemos uso de esta librería
#!pip install anndata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-19 22:47:25.125048: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.3 MB 194 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-19 22:47:41.326883: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi

# Código de Oswaldo `parserBIO_v3_may8_2021.py`
Adaptamos el código que nos dejo Oswaldo

In [ ]:
# Cargamos las librerías necesarias
import spacy
import os
import re
#import en_core_web_sm
import es_core_news_md #Importante que sea el espanol pues los textos están escritos en castellano
import es_core_news_sm
import es_core_news_lg

#nlp = en_core_web_sm.load()

nlp = es_core_news_md.load()

BEGIN = 'B'
INSIDE = 'I'
OUTSIDE = 'O'

def getDictWords(text):
	words = {}
	ent = nlp(text)
	for i in range(len(ent)):
		words[ent[i].idx] = ent[i].text
    	
	return words

def getDictEntities(file_ann):
  entities = {}
  with open(file_ann) as anns:
    for ann in anns:

      start = int(ann[:-1].split('\t')[1].split(' ')[1])
      end = int(ann[:-1].split('\t')[1].split(' ')[2])

      entities[(start, end)] = [ann[:-1].split('\t')[2],ann.split('\t')[1].split(' ')[0]]

  return entities

def is_empty(string): 
	return not string.strip() 


def getBIO(words,entities):
	bio = []
	keys = words.keys()
	found = False
	processedConcepts = []

	for key in keys:
		if(not words[key] in processedConcepts):
			keysEntities = entities.keys()
			for keyEntity in keysEntities:
				start = keyEntity[0]
				end = keyEntity[1]
				if(key==start):
					concept = entities[keyEntity]
					if(len(concept[0])==1):
						bio.append(concept[0]+"\t"+BEGIN+"_"+concept[1]+"\n")
					else:
						ent = nlp(concept[0])
						bio.append(ent[0].text+"\t"+BEGIN+"_"+concept[1]+"\n")

						for i in range(len(ent)-1):
							spaces = (ent[i + 1].idx) - (ent[i].idx + len(ent[i].text))
							start = start + len(ent[i].text) + spaces
							bio.append(ent[i+1].text+"\t"+INSIDE+"_"+concept[1]+"\n")
							processedConcepts.append(ent[i+1].text)
						found = True

			if (not found):
				if(not(is_empty(words[key]))):
					bio.append(words[key]+"\t"+OUTSIDE+"\n")
					
					if("." == words[key]):
						bio.append("\n")
		
				else:
					bio.append("\n")
		
			found = False
		else:
			processedConcepts.remove(words[key])
		
	return bio

def bioToFile(pathOutput,bio):
	f = open(pathOutput,"w")
	for i in range(len(bio)):
		f.write(bio[i])
	f.close()


def readFile(pathFile):
	f = open(pathFile,'r') 
	text = f.read()
	f.close()
	return text	

# Lo hacemos con md

# Se eligen las carpetas que contienen tanto los archivos en .txt como las anotaciones .ann 

# Conjunto de entrenamiento
# TASK 1
#path = '/content/drive/MyDrive/Colab Notebooks/Corpus/Original/Train/task1'
#path_Output = '/content/drive/MyDrive/Colab Notebooks/Corpus/Procesado/Oswaldo/Train/task1converted/'

# TASK 2
#path = '/content/drive/MyDrive/Colab Notebooks/Corpus/Original/Train/task2'
#path_Output = '/content/drive/MyDrive/Colab Notebooks/Corpus/Procesado/Oswaldo/Train/task2converted/'

# Conjunto de test
# TASK 1
#path = '/content/drive/MyDrive/Colab Notebooks/Corpus/Original/Test/ner'
#path_Output = '/content/drive/MyDrive/Colab Notebooks/Corpus/Procesado/Oswaldo/Test/nerconverted/'

# TASK2
#path = '/content/drive/MyDrive/Colab Notebooks/Corpus/Original/Test/class'
#path_Output = '/content/drive/MyDrive/Colab Notebooks/Corpus/Procesado/Oswaldo/Test/classconverted/'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
	for file in f:
		if file.endswith(".txt"):
			files.append(os.path.join(r, file))

	for file in files:
		text = readFile(file)
		words = getDictWords(text)
		entities = getDictEntities(file.replace(".txt",".ann"))
		bio = getBIO(words,entities)
		head, tail = os.path.split(file)
		bioToFile(path_Output+tail.replace(".txt",".bio"),bio)

# Código Neuroner 
Finalmente optamos por usar el código de NeuroNER

https://github.com/Franck-Dernoncourt/NeuroNER

In [ ]:
!pip install pycorenlp
!pip3 install pyneuroner[cpu]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pycorenlp: filename=pycorenlp-0.3.0-py3-none-any.whl size=2144 sha256=d7c8392fe1b3913ccc54fc224e91610d69d4650630aac41955efa17ea8532fb8
  Stored in directory: /root/.cache/pip/wheels/2e/65/32/176699d5db77f83d0510bd5ce6a5a41397125ce84f9c75e329
Successfully built pycorenlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.9 MB 17.8 MB/s 


In [ ]:
# -*- coding: utf-8 -*-
import codecs
import glob
import json
import os

from pycorenlp import StanfordCoreNLP
import spacy

from neuroner import utils_nlp

def get_start_and_end_offset_of_token_from_spacy(token):
    start = token.idx
    end = start + len(token)
    return start, end

def get_sentences_and_tokens_from_spacy(text, spacy_nlp):
    document = spacy_nlp(text)
    # sentences
    sentences = []
    for span in document.sents:
        sentence = [document[i] for i in range(span.start, span.end)]
        sentence_tokens = []
        for token in sentence:
            token_dict = {}
            token_dict['start'], token_dict['end'] = get_start_and_end_offset_of_token_from_spacy(token)
            token_dict['text'] = text[token_dict['start']:token_dict['end']]
            if token_dict['text'].strip() in ['\n', '\t', ' ', '']:
                continue
            # Make sure that the token text does not contain any space
            if len(token_dict['text'].split(' ')) != 1:
                print("WARNING: the text of the token contains space character, replaced with hyphen\n\t{0}\n\t{1}".format(token_dict['text'], 
                                                                                                                           token_dict['text'].replace(' ', '-')))
                token_dict['text'] = token_dict['text'].replace(' ', '-')
            sentence_tokens.append(token_dict)
        sentences.append(sentence_tokens)
    return sentences

def get_stanford_annotations(text, core_nlp, port=9000, annotators='tokenize,ssplit,pos,lemma'):
    output = core_nlp.annotate(text, properties={
        "timeout": "10000",
        "ssplit.newlineIsSentenceBreak": "two",
        'annotators': annotators,
        'outputFormat': 'json'
    })
    if type(output) is str:
        output = json.loads(output, strict=False)
    return output

def get_sentences_and_tokens_from_stanford(text, core_nlp):
    stanford_output = get_stanford_annotations(text, core_nlp)
    sentences = []
    for sentence in stanford_output['sentences']:
        tokens = []
        for token in sentence['tokens']:
            token['start'] = int(token['characterOffsetBegin'])
            token['end'] = int(token['characterOffsetEnd'])
            token['text'] = text[token['start']:token['end']]
            if token['text'].strip() in ['\n', '\t', ' ', '']:
                continue
            # Make sure that the token text does not contain any space
            if len(token['text'].split(' ')) != 1:
                print("WARNING: the text of the token contains space character, replaced with hyphen\n\t{0}\n\t{1}".format(token['text'], 
                                                                                                                           token['text'].replace(' ', '-')))
                token['text'] = token['text'].replace(' ', '-')
            tokens.append(token)
        sentences.append(tokens)
    return sentences

def get_entities_from_brat(text_filepath, annotation_filepath, verbose=False):
    # load text
    with codecs.open(text_filepath, 'r', 'UTF-8') as f:
        text =f.read()
    if verbose: print("\ntext:\n{0}\n".format(text))

    # parse annotation file
    entities = []
    with codecs.open(annotation_filepath, 'r', 'UTF-8') as f:
        for line in f.read().splitlines():
            anno = line.split()
            id_anno = anno[0]
            # parse entity
            if id_anno[0] == 'T':
                entity = {}
                entity['id'] = id_anno
                entity['type'] = anno[1]
                entity['start'] = int(anno[2])
                entity['end'] = int(anno[3])
                entity['text'] = ' '.join(anno[4:])
                if verbose:
                    print("entity: {0}".format(entity))
                # Check compatibility between brat text and anootation
                if utils_nlp.replace_unicode_whitespaces_with_ascii_whitespace(text[entity['start']:entity['end']]) != \
                    utils_nlp.replace_unicode_whitespaces_with_ascii_whitespace(entity['text']):
                    print("Warning: brat text and annotation do not match.")
                    print("\ttext: {0}".format(text[entity['start']:entity['end']]))
                    print("\tanno: {0}".format(entity['text']))
                # add to entitys data
                entities.append(entity)
    if verbose: print("\n\n")
    
    return text, entities

def check_brat_annotation_and_text_compatibility(brat_folder):
    '''
    Check if brat annotation and text files are compatible.
    '''
    dataset_type =  os.path.basename(brat_folder)
    print("Checking the validity of BRAT-formatted {0} set... ".format(dataset_type), end='')
    text_filepaths = sorted(glob.glob(os.path.join(brat_folder, '*.txt')))
    for text_filepath in text_filepaths:
        base_filename = os.path.splitext(os.path.basename(text_filepath))[0]
        annotation_filepath = os.path.join(os.path.dirname(text_filepath), base_filename + '.ann')
        # check if annotation file exists
        if not os.path.exists(annotation_filepath):
            raise IOError("Annotation file does not exist: {0}".format(annotation_filepath))
        text, entities = get_entities_from_brat(text_filepath, annotation_filepath)
    print("Done.")

def brat_to_conll(input_folder, output_filepath, tokenizer, language):
    '''
    Assumes '.txt' and '.ann' files are in the input_folder.
    Checks for the compatibility between .txt and .ann at the same time.
    '''
    if tokenizer == 'spacy':
        spacy_nlp = spacy.load(language)
    elif tokenizer == 'stanford':
        core_nlp = StanfordCoreNLP('http://localhost:{0}'.format(9000))
    else:
        raise ValueError("tokenizer should be either 'spacy' or 'stanford'.")
    verbose = False
    dataset_type =  os.path.basename(input_folder)
    print("Formatting {0} set from BRAT to CONLL... ".format(dataset_type), end='')
    text_filepaths = sorted(glob.glob(os.path.join(input_folder, '*.txt')))
    output_file = codecs.open(output_filepath, 'w', 'utf-8')
    for text_filepath in text_filepaths:
        base_filename = os.path.splitext(os.path.basename(text_filepath))[0]
        annotation_filepath = os.path.join(os.path.dirname(text_filepath), base_filename + '.ann')
        # create annotation file if it does not exist
        if not os.path.exists(annotation_filepath):
            codecs.open(annotation_filepath, 'w', 'UTF-8').close()

        text, entities = get_entities_from_brat(text_filepath, annotation_filepath)
        entities = sorted(entities, key=lambda entity:entity["start"])
        
        if tokenizer == 'spacy':
            sentences = get_sentences_and_tokens_from_spacy(text, spacy_nlp)
        elif tokenizer == 'stanford':
            sentences = get_sentences_and_tokens_from_stanford(text, core_nlp)
        
        for sentence in sentences:
            inside = False
            previous_token_label = 'O'
            for token in sentence:
                token['label'] = 'O'
                for entity in entities:
                    if entity['start'] <= token['start'] < entity['end'] or \
                       entity['start'] < token['end'] <= entity['end'] or \
                       token['start'] < entity['start'] < entity['end'] < token['end']:

                        token['label'] = entity['type'].replace('-', '_') # Because the ANN doesn't support tag with '-' in it

                        break
                    elif token['end'] < entity['start']:
                        break
                        
                if len(entities) == 0:
                    entity={'end':0}
                if token['label'] == 'O':
                    gold_label = 'O'
                    inside = False
                elif inside and token['label'] == previous_token_label:
                    gold_label = 'I-{0}'.format(token['label'])
                else:
                    inside = True
                    gold_label = 'B-{0}'.format(token['label'])
                if token['end'] == entity['end']:
                    inside = False
                previous_token_label = token['label']
                if verbose: print('{0} {1} {2} {3} {4}\n'.format(token['text'], base_filename, token['start'], token['end'], gold_label))
                output_file.write('{0} {1} {2} {3} {4}\n'.format(token['text'], base_filename, token['start'], token['end'], gold_label))
            if verbose: print('\n')
            output_file.write('\n')

    output_file.close()
    print('Done.')
    if tokenizer == 'spacy':
        del spacy_nlp
    elif tokenizer == 'stanford':
        del core_nlp

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


Las anotaciones se convierten en un único archivo

In [ ]:
# Poner el tokenizador en espanol

brat_to_conll("/content/drive/MyDrive/Colab Notebooks/Corpus/Original/Train/task1", "task1neuronersm.txt", "spacy", "es_core_news_sm")
brat_to_conll("/content/drive/MyDrive/Colab Notebooks/Corpus/Original/Train/task2", "task2neuronersm.txt", "spacy", "es_core_news_sm")
brat_to_conll("/content/drive/MyDrive/Colab Notebooks/Corpus/Original/Test/class", "classneuronersm.txt", "spacy", "es_core_news_sm")
brat_to_conll("/content/drive/MyDrive/Colab Notebooks/Corpus/Original/Test/ner", "nerneuronersm.txt", "spacy", "es_core_news_sm")

#brat_to_conll("/content/drive/MyDrive/Colab Notebooks/Corpus/Original/Train/task1", "task1neuroner.txt", "spacy", "es_core_news_md")
#brat_to_conll("/content/drive/MyDrive/Colab Notebooks/Corpus/Original/Train/task2", "task2neuroner.txt", "spacy", "es_core_news_md")
#brat_to_conll("/content/drive/MyDrive/Colab Notebooks/Corpus/Original/Test/class", "classneuroner.txt", "spacy", "es_core_news_md")
#brat_to_conll("/content/drive/MyDrive/Colab Notebooks/Corpus/Original/Test/ner", "nerneuroner.txt", "spacy", "es_core_news_md")

#brat_to_conll("/content/drive/MyDrive/Colab Notebooks/Corpus/Original/Train/task1", "task1neuronersm.txt", "spacy", "es_core_news_lg")
#brat_to_conll("/content/drive/MyDrive/Colab Notebooks/Corpus/Original/Train/task2", "task2neuronersm.txt", "spacy", "es_core_news_lg")
#brat_to_conll("/content/drive/MyDrive/Colab Notebooks/Corpus/Original/Test/class", "classneuronersm.txt", "spacy", "es_core_news_lg")
#brat_to_conll("/content/drive/MyDrive/Colab Notebooks/Corpus/Original/Test/ner", "nerneuronersm.txt", "spacy", "es_core_news_lg")

Formatting task1 set from BRAT to CONLL... WARNING: the text of the token contains space character, replaced with hyphen
	EE. UU.
	EE.-UU.
	EE. UU.
	EE.-UU.
	EE. UU.
	EE.-UU.
Done.
Formatting task2 set from BRAT to CONLL... WARNING: the text of the token contains space character, replaced with hyphen
	EE. UU.
	EE.-UU.
	EE. UU.
	EE.-UU.
	EE. UU.
	EE.-UU.
Done.
Formatting class set from BRAT to CONLL... Done.
Formatting ner set from BRAT to CONLL... Done.


Para juntar todos los archivos .ann en uno solo:

En la terminal, en MacOS o UNIX nos vamos a la carpeta donde estén los archivos .ann (ejemplo: task1) y lanzamos el siguiente comando

https://unix.stackexchange.com/questions/387377/merge-file-text-with-file-name

`awk '{print $0,FILENAME}' *.ann >> ~/Desktop/nombredelarchivo.txt`

Para instalar awk en MacOS podemos acudir [aquí](https://stackoverflow.com/questions/24332942/why-awk-script-does-not-work-on-mac-os-but-works-on-linux)